In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/pbig5min.dat' #Ubuntu
path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/pbig1min.dat' #Server

#read the REPPU data
with open (path) as f:
    rectype = np.dtype(np.float32)
    reppu_data = np.fromfile(f, rectype) #size = 109,900,800

reppu_data = reppu_data.reshape(-1,30,80)
reppu_data.shape

(228960, 30, 80)

In [5]:
#Read MHD dates and expand-out the date ranges
mhd_data = pd.read_csv('mhd_dates.csv')
expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
    for start, end in zip(mhd_data['start'], mhd_data['end'])])

In [9]:
time_res = 1

if time_res == 1:
    time_step = 1440
else :
    time_step = 288

In [10]:
data_reshaped = reppu_data.reshape(len(expanded_dt), time_step, 30, 80) 

# Define coordinates
time = np.arange(time_step)
lat = np.linspace(53.1, 89.7, 30) #30 intervals between 53.1° to 89.7°
lon = np.linspace(1.6, 357.6, 80) #80 intervals between 1.6° to 357.6°

# Create 'dt' variable combining dates and five-minute intervals
dt = []
for day in expanded_dt:
    for t in time:
        dt.append(day + pd.Timedelta(minutes=t*time_res))
dt = np.array(dt) #convert from list to numpy array

# Create xarray Dataset
ds = xr.Dataset({'potential': (['dt', 'lat', 'lon'], data_reshaped.reshape(-1, 30, 80))},
coords={'dt': dt, 'lat': lat, 'lon': lon})

# Add potential and units
ds['potential'].attrs['units'] = 'kV'
ds['potential'] = ds['potential'] * 1e-3 # Convert to kV
ds

<xarray.Dataset>
Dimensions:    (dt: 228960, lat: 30, lon: 80)
Coordinates:
  * dt         (dt) datetime64[ns] 2021-05-10 ... 2022-07-31T23:59:00
  * lat        (lat) float64 53.1 54.36 55.62 56.89 ... 85.91 87.18 88.44 89.7
  * lon        (lon) float64 1.6 6.106 10.61 15.12 ... 344.1 348.6 353.1 357.6
Data variables:
    potential  (dt, lat, lon) float32 1.279 1.026 0.7595 ... -23.67 -23.72

In [21]:
omni_df = pd.read_csv('omni_mhd_5min.csv')
#omni_df = pd.read_csv(omni_mhd_path+'omni_mhd_5min.csv')
omni_df.set_index('dt', inplace=True) #set the datetime as the index
omni_df = omni_df.ffill().bfill()
omni_df = omni_df.dropna() #drop any remaining NaNs
omni_df

omni_ds = xr.Dataset(omni_df)
omni_ds['dt'] = pd.to_datetime(omni_ds['dt']) #convert the index to datetime

#merge OMNI with REPPU data
reppu_omni_ds = ds.merge(omni_ds, join='outer')

#select date range
reppu_omni_ds = reppu_omni_ds.sortby('dt')
reppu_omni_ds

#interpolate the missing values in BY_GSE
reppu_omni_ds['BY_GSE'] = reppu_omni_ds['BY_GSE'].interpolate_na(dim='dt')
reppu_omni_ds['BZ_GSE'] = reppu_omni_ds['BZ_GSE'].interpolate_na(dim='dt')
reppu_omni_ds['flow_speed'] = reppu_omni_ds['flow_speed'].interpolate_na(dim='dt')
reppu_omni_ds['proton_density'] = reppu_omni_ds['proton_density'].interpolate_na(dim='dt')
reppu_omni_ds['tilt_angle'] = reppu_omni_ds['tilt_angle'].interpolate_na(dim='dt')

#drop nan values
reppu_omni_ds = reppu_omni_ds.dropna('dt')
reppu_omni_ds

<xarray.Dataset>
Dimensions:         (dt: 228956, lat: 30, lon: 80)
Coordinates:
  * dt              (dt) datetime64[ns] 2021-05-10 ... 2022-08-19T23:55:00
  * lat             (lat) float64 53.1 54.36 55.62 56.89 ... 87.18 88.44 89.7
  * lon             (lon) float64 1.6 6.106 10.61 15.12 ... 348.6 353.1 357.6
Data variables:
    potential       (dt, lat, lon) float32 1.279 1.026 0.7595 ... -12.47 -12.5
    BY_GSE          (dt) float64 4.72 4.646 4.572 4.498 ... -1.968 -1.944 -1.92
    BZ_GSE          (dt) float64 6.24 6.302 6.364 6.426 ... -4.556 -4.528 -4.5
    flow_speed      (dt) float64 344.0 344.0 344.0 344.0 ... 656.0 657.0 658.0
    proton_density  (dt) float64 12.11 11.98 11.84 11.71 ... 5.49 5.44 5.39 5.34
    tilt_angle      (dt) float64 0.253 0.2524 0.2517 ... 0.1748 0.1742 0.1735

In [ ]:
#reppu_omni_ds_range1 = reppu_omni_ds.sel(dt=slice('2021-12-01', '2022-01-24'))
#reppu_omni_ds_range2 = reppu_omni_ds.sel(dt=slice('2022-06-10', '2022-07-31'))
#reppu_omni_ds_combined = xr.concat([reppu_omni_ds_range1, reppu_omni_ds_range2], dim='dt')
#reppu_omni_ds = reppu_omni_ds_combined.sortby('dt')
#reppu_omni_ds

5 min slices

In [ ]:
#train_slice = slice(0, 15840) #55days * 288 = 15840
#test_slice = slice(15840, None) #15 days * 288 = 4320

#train_slice = slice(0, 11520) #40days * 288 = 11520
#test_slice = slice(11520, None) #10 days * 288 = 2880 

# Define the slice ranges for train and test data
train_slice = slice(0, 36576) #36576 / 24 / 12 = 127 days = 80% of the data
test_slice = slice(36576, None) # 9216 / 24 / 12 = 32 days = 20% of the data


1 min slices

In [22]:
train_slice = slice(0, 182880) #36576 / 24 / 12 = 127 days = 80% of the data
test_slice = slice(182880, None) # 9216 / 24 / 12 = 32 days = 20% of the data

In [26]:

ds_train = reppu_omni_ds.isel(dt=train_slice)
ds_test = reppu_omni_ds.isel(dt=test_slice)

X_train = ds_train.drop_vars('potential').to_array().values.T
y_train = ds_train['potential'].values
y_train = y_train.reshape(-1, 30*80)
#y_train = np.mean(y_train, axis=1)

X_test = ds_test.drop_vars('potential').to_array().values.T
y_test = ds_test['potential'].values
y_test = y_test.reshape(-1, 30*80)
#y_test = np.mean(y_test, axis=1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((182880, 5), (182880, 2400), (46076, 5), (46076, 2400))

In [27]:
# Normalizing the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [28]:
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

In [29]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        # Remove extra dimensions from output
        out = self.fc(out[:, -1, :].squeeze())  # Squeeze the output
        return out


In [30]:
def dummy_slider():
    x = np.arange(0, 288)
    seq_len = 12
    slider = 6

    for i in range(0, len(x) - seq_len + 1, slider):
        inputs = x[i:i + seq_len]
        targets = x[i + seq_len - 1]
     
        print(f'inputs: {inputs}, targets: {targets}')

dummy_slider()

inputs: [ 0  1  2  3  4  5  6  7  8  9 10 11], targets: 11
inputs: [ 6  7  8  9 10 11 12 13 14 15 16 17], targets: 17
inputs: [12 13 14 15 16 17 18 19 20 21 22 23], targets: 23
inputs: [18 19 20 21 22 23 24 25 26 27 28 29], targets: 29
inputs: [24 25 26 27 28 29 30 31 32 33 34 35], targets: 35
inputs: [30 31 32 33 34 35 36 37 38 39 40 41], targets: 41
inputs: [36 37 38 39 40 41 42 43 44 45 46 47], targets: 47
inputs: [42 43 44 45 46 47 48 49 50 51 52 53], targets: 53
inputs: [48 49 50 51 52 53 54 55 56 57 58 59], targets: 59
inputs: [54 55 56 57 58 59 60 61 62 63 64 65], targets: 65
inputs: [60 61 62 63 64 65 66 67 68 69 70 71], targets: 71
inputs: [66 67 68 69 70 71 72 73 74 75 76 77], targets: 77
inputs: [72 73 74 75 76 77 78 79 80 81 82 83], targets: 83
inputs: [78 79 80 81 82 83 84 85 86 87 88 89], targets: 89
inputs: [84 85 86 87 88 89 90 91 92 93 94 95], targets: 95
inputs: [ 90  91  92  93  94  95  96  97  98  99 100 101], targets: 101
inputs: [ 96  97  98  99 100 101 102 103 10

In [31]:
seq_len = 5
slider = 30

# Instantiate the LSTM model with updated input_size
input_size = X_train_tensor.shape[1]
hidden_size = 64
num_layers = 2
output_size = y_train_tensor.shape[1]
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [32]:
#Train the model

num_epochs = 50
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor) - seq_len + 1, slider):
        inputs = X_train_tensor[i:i + seq_len].to(device)
        targets = y_train_tensor[i + seq_len - 1].to(device)
        # Forward pass
        outputs = model(inputs.unsqueeze(0)) # Add batch dimension
        loss = criterion(outputs, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    rmse = np.sqrt(loss.item())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.2f}, RMSE: {rmse:.2f} V')


Epoch [1/50], Loss: 76.73, RMSE: 8.76 V
Epoch [2/50], Loss: 66.43, RMSE: 8.15 V
Epoch [3/50], Loss: 67.61, RMSE: 8.22 V
Epoch [4/50], Loss: 58.88, RMSE: 7.67 V
Epoch [5/50], Loss: 74.13, RMSE: 8.61 V
Epoch [6/50], Loss: 91.81, RMSE: 9.58 V
Epoch [7/50], Loss: 75.74, RMSE: 8.70 V
Epoch [8/50], Loss: 72.47, RMSE: 8.51 V
Epoch [9/50], Loss: 66.43, RMSE: 8.15 V
Epoch [10/50], Loss: 69.13, RMSE: 8.31 V
Epoch [11/50], Loss: 94.19, RMSE: 9.71 V
Epoch [12/50], Loss: 78.68, RMSE: 8.87 V
Epoch [13/50], Loss: 65.98, RMSE: 8.12 V
Epoch [14/50], Loss: 69.86, RMSE: 8.36 V
Epoch [15/50], Loss: 103.26, RMSE: 10.16 V
Epoch [16/50], Loss: 94.95, RMSE: 9.74 V
Epoch [17/50], Loss: 104.98, RMSE: 10.25 V
Epoch [18/50], Loss: 70.31, RMSE: 8.39 V
Epoch [19/50], Loss: 78.98, RMSE: 8.89 V
Epoch [20/50], Loss: 89.91, RMSE: 9.48 V
Epoch [21/50], Loss: 94.51, RMSE: 9.72 V
Epoch [22/50], Loss: 135.85, RMSE: 11.66 V
Epoch [23/50], Loss: 93.32, RMSE: 9.66 V
Epoch [24/50], Loss: 80.75, RMSE: 8.99 V
Epoch [25/50], Loss

KeyboardInterrupt: 

In [ ]:
model.eval()

predicted = []

with torch.no_grad():
    for i in range(0, len(X_test_tensor) - seq_len + 1, slider):
        inputs = X_test_tensor[i:i + seq_len].to(device)  # Move inputs to GPU
        outputs = model(inputs.unsqueeze(0))
        predicted.append(outputs)

predicted = torch.cat(predicted, dim=0)
predicted = predicted.cpu().numpy() # Move to CPU and convert to numpy array

In [ ]:
model_cpu = model.to('cpu')
torch.save(model_cpu.state_dict(), 'lstm_model.pth')

In [ ]:
dt = ds_test['dt'].values
dt = dt[seq_len - 1::slider]

predicted_ds = xr.Dataset({'predicted_pot': (['dt', 'lat', 'lon'], predicted.reshape(-1, 30, 80))},
                            coords={'dt': dt, 'lat': lat, 'lon': lon})

predicted_ds = xr.merge([ds_test, predicted_ds], join='inner')
predicted_ds['RMSE'] = np.sqrt((predicted_ds['predicted_pot'] - predicted_ds['potential'])**2)
predicted_ds['RMSE'].attrs['units'] = 'kV'

predicted_ds

In [ ]:
np.mean(predicted_ds['RMSE']).values

In [ ]:
fig = plt.figure(figsize=(6,3))

#plot RMSE as radar plot
cbar = predicted_ds['RMSE'].mean('dt').plot()
plt.title('RMSE of Polar Cap Potential')
plt.ylabel('Magnetic Latitude [deg]')
#plt.xticks(np.arange(0, 25, 3))
plt.xlabel('MLT')

plt.tight_layout()

In [ ]:
date = '2022-07-05T00:10:00.000000000'

# Extract true and predicted values
pot_true = predicted_ds['potential'].sel(dt=date).values
pot_pred = predicted_ds['predicted_pot'].sel(dt=date).values

pot_true


RMSE

In [ ]:
date = '2022-07-05T00:10:00.000000000'

# Extract true and predicted values
pot_true = predicted_ds['potential'].sel(dt=date).values
pot_pred = predicted_ds['predicted_pot'].sel(dt=date).values
rmse  = predicted_ds['RMSE'].sel(dt=date).values

# Calculate the difference
diff = pot_true - pot_pred

# Set up the plot
fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(14, 4))

# Define common parameters
theta = np.linspace(0, 360, 80) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(53.1, 89.7, 30)
shrink = .3

# Plot true values
c_true = axs[0].contourf(theta_rad, r, -pot_true, levels=np.linspace(-30, 30, 50), cmap='PuOr', extend='both')
axs[0].set_ylim([0, 30])
axs[0].set_yticks([0, 10, 20, 30])
axs[0].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[0].set_xlim([-np.pi, np.pi])
axs[0].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[0].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[0].set_title(f'True Potential \n (REPPU)')
plt.colorbar(c_true, ax=axs[0], label='Potential [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot predicted values
c_pred = axs[1].contourf(theta_rad, r, -pot_pred, levels=np.linspace(-30, 30, 50), cmap='PuOr', extend='both')
axs[1].set_ylim([0, 30])
axs[1].set_yticks([0, 10, 20, 30])
axs[1].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[1].set_xlim([-np.pi, np.pi])
axs[1].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[1].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[1].set_title(f'Predicted Potential \n (SMRAI)')
plt.colorbar(c_pred, ax=axs[1], label='Potential [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot difference
c_diff = axs[2].contourf(theta_rad, r, rmse, levels=np.linspace(0, 8, 50), cmap='Reds', extend='both')
axs[2].set_ylim([0, 30])
axs[2].set_yticks([0, 10, 20, 30])
axs[2].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[2].set_xlim([-np.pi, np.pi])
axs[2].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[2].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[2].set_title('RMSE')
plt.colorbar(c_diff, ax=axs[2], label='RMSE [kV]', shrink=shrink, pad=0.1, ticks=np.arange(0,9,1), orientation='horizontal')


plt.subplots_adjust(wspace=-0.7)  # Adjust horizontal space between subplots

plt.tight_layout()
plt.show()

Difference

In [ ]:
date = '2022-07-05T00:10:00.000000000'

# Extract true and predicted values
pot_true = predicted_ds['potential'].sel(dt=date).values
pot_pred = predicted_ds['predicted_pot'].sel(dt=date).values
rmse  = predicted_ds['RMSE'].sel(dt=date).values

# Calculate the difference
diff = pot_true - pot_pred

# Set up the plot
fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(14, 4))

# Define common parameters
theta = np.linspace(0, 360, 80) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(53.1, 89.7, 30)
shrink = .3

# Plot true values
c_true = axs[0].contourf(theta_rad, r, -pot_true, levels=np.linspace(-30, 30, 50), cmap='PuOr', extend='both')
axs[0].set_ylim([0, 30])
axs[0].set_yticks([0, 10, 20, 30])
axs[0].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[0].set_xlim([-np.pi, np.pi])
axs[0].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[0].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[0].set_title(f'True Potential \n (REPPU)')
plt.colorbar(c_true, ax=axs[0], label='Potential [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot predicted values
c_pred = axs[1].contourf(theta_rad, r, -pot_pred, levels=np.linspace(-30, 30, 50), cmap='PuOr', extend='both')
axs[1].set_ylim([0, 30])
axs[1].set_yticks([0, 10, 20, 30])
axs[1].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[1].set_xlim([-np.pi, np.pi])
axs[1].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[1].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
axs[1].set_title(f'Predicted Potential \n (SMRAI)')
plt.colorbar(c_pred, ax=axs[1], label='Potential [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

# Plot difference
c_diff = axs[2].contourf(theta_rad, r, rmse, levels=np.linspace(-30, 30, 50), cmap='bwr', extend='both')
axs[2].set_ylim([0, 30])
axs[2].set_yticks([0, 10, 20, 30])
axs[2].set_yticklabels(["90°", "80°", "70°", "60° MLAT"])
axs[2].set_xlim([-np.pi, np.pi])
axs[2].set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
axs[2].set_xticklabels(["21", "0 MLT", "3", "6", "9", "12", "15", "18"])
#axs[2].set_title('Difference \n(True - Predicted)')
#plt.colorbar(c_diff, ax=axs[2], label='Difference [kV]', shrink=shrink, pad=0.1, ticks=np.arange(-30, 31, 10), orientation='horizontal')

#for RMAE
axs[2].set_title('RMSE')
plt.colorbar(c_diff, ax=axs[2], label='RMSE [kV]', shrink=shrink, pad=0.1, ticks=np.arange(0,31, 10), orientation='horizontal')


plt.subplots_adjust(wspace=-0.7)  # Adjust horizontal space between subplots

plt.tight_layout()
plt.show()